In [2]:
import networkx as nx
import pickle

import numpy as np; np.set_printoptions(suppress=True, linewidth=180); np.seterr(divide='ignore')
import pandas as pd; pd.set_option('display.width', 500)
import matplotlib.pyplot as plt

from plot_module import plot_ode_results, plot_sde_results, plot_simulator_graph, plot_neuron_graph_subset, plot_twocomponent_results
from sim_param_from_network import names_from_network, sde_param_from_network, ode_from_network, start_state_from_nodes
from neuron_graph_process import neuron_graph_process
from neuron_graph_helper import load_pickled_neuron_graph
from analyse_simulation import two_component_statistics

from simulate import simulate_ode, simulate_gillespie, simulate_tauleaping

In [3]:
G = load_pickled_neuron_graph('neuron_graphs/2_compartment_model.pkl')

In [8]:
DELTA = 0.25
G.nodes()['S0B']['delta'] = DELTA

VARS, NODES = names_from_network(G)
START_STATE = start_state_from_nodes(G, heteroplasmy=0.5, delta=DELTA)


C_B_val = sequence = [round(i * 10**-decimals, 14) for decimals in range(1, 13) for i in range(10, 0, -1)]

C_B_val = [element for i, element in enumerate(C_B_val) if i % 5 == 0] # sparseify for testing
C_B_val

[1.0,
 0.5,
 0.1,
 0.05,
 0.01,
 0.005,
 0.001,
 0.0005,
 0.0001,
 5e-05,
 1e-05,
 5e-06,
 1e-06,
 5e-07,
 1e-07,
 5e-08,
 1e-08,
 5e-09,
 1e-09,
 5e-10,
 1e-10,
 5e-11,
 1e-11,
 5e-12]

In [9]:
TIME_POINTS = np.linspace(0, 4000, 1001)
REP = 60


stats_df = pd.DataFrame()

for c_b in C_B_val:
    print(f"\n<<<< STARTING SIMULATION WITH C_B = {c_b} >>>>\n")
    
    G.nodes()['S0B']['c_b'] = c_b

#     # infer the ode model
#     ode_model = ode_from_network(G, prnt=True)

#     # run the ode model
#     ode_results = simulate_ode(ode_model, TIME_POINTS, START_STATE)
#     plot_ode_results(ode_results, TIME_POINTS, DELTA, VARS, NODES, prnt=False)

    # infer the parameterse of the sde systems
    SDE_PARAM = sde_param_from_network(G, prnt=False)

    # run the gillespie simulation
    gillespie_results = simulate_gillespie(SDE_PARAM, TIME_POINTS, START_STATE, replicates=REP)
    df_g, stats_g = two_component_statistics(gillespie_results, TIME_POINTS, DELTA)
    
    stats_g['simtype'] = 'g'
    stats_g['cb'] = c_b
    stats_df = stats_df.append(pd.Series(stats_g), ignore_index=True)
    
    print(df_g.iloc[[0, -1]]);print()
    #plot_twocomponent_results(df_g)
    
    
    # run the tau leaping simulation
    tauleaping_results = simulate_tauleaping(SDE_PARAM, TIME_POINTS, START_STATE, replicates=REP, timestep=0.005)
    df_t, stats_t = two_component_statistics(tauleaping_results, TIME_POINTS, DELTA)
    
    stats_t['simtype'] = 't'
    stats_t['cb'] = c_b
    stats_df = stats_df.append(pd.Series(stats_t), ignore_index=True)
    
    print(df_t.iloc[[0, -1]]);print()
    #plot_twocomponent_results(df_t)
    
    print("\n ----- \n")
    
print(stats_df)


<<<< STARTING SIMULATION WITH C_B = 1.0 >>>>

simulating using gillespie...


100%|██████████| 60/60 [00:10<00:00,  5.54it/s]



Copy number:
wilcox. test: greater p= 0.0
pooled t test: greater p= 0.0036
mk trend: increasing p= 0.0
kpss: non-stationary p= 0.01

Eff. pop. size:
wilcox. test: none p= 0.6758
pooled t test: none p= 0.1038
mk trend: no trend p= 0.9732
kpss: stationary p= 0.1

Heteroplasmy:
wilcox. test: greater p= 0.0
pooled t test: greater p= 0.0153
mk trend: increasing p= 0.0
kpss: non-stationary p= 0.01

           t  ps_mean  ps_sem  eps_mean  eps_sem  het_mean  het_sem  p_het_1  p_het_0  p_pop0  s_wt_mean  s_wt_sem  s_mt_mean  s_mt_sem  a_wt_mean  a_wt_sem  a_mt_mean  a_mt_sem
0        0.0   640.00     0.0    400.00      0.0    0.5000   0.0000   0.0000   0.0000     0.0     160.00      0.00     160.00      0.00     160.00      0.00      160.0      0.00
1000  4000.0  1320.28    65.4    400.57      1.0    0.7667   0.0546   0.7667   0.2333     0.0      46.73     10.94     612.98     43.66      47.27     11.08      613.3     43.77

simulating using tau leaping...


 60%|██████    | 36/60 [00:09<00:04,  5.95it/s]

KeyboardInterrupt: 